In [3]:
# | default_exp configs.prompts

In [4]:
# | export
import os
import platform
from pathlib import Path

In [5]:
# | export

def get_system_context():
    """Generate enhanced system context with detailed environment information"""
    
    # Basic system info
    system_info = {
        "os": platform.system(),
        "os_version": platform.version(),
        "architecture": platform.machine(),
        "python_version": platform.python_version(),
    "current_directory": str(Path.cwd()),
    "home_directory": str(Path.home()),
    }
    
    # Project context detection
    cwd = Path.cwd()
    project_indicators = {
        'git_repo': (cwd / '.git').exists(),
        'python_project': any((cwd / f).exists() for f in [".git",'setup.py', 'pyproject.toml', 'requirements.txt']),
        'nbdev_project': (cwd / 'settings.ini').exists() and (cwd / 'nbs').exists(),
        'node_project': (cwd / 'package.json').exists(),
        'docker_project': any((cwd / f).exists() for f in ['Dockerfile', 'docker-compose.yml']),
    }
    
    system_info['project_context'] = project_indicators
    return system_info


def get_system_prompt():
    """Generate production-grade system prompt for Buddy AI"""
    
    system_context = get_system_context()
    project_types = [k.replace('_project', '') for k, v in system_context['project_context'].items() if v]
    project_context_str = f"Detected: {', '.join(project_types)}" if project_types else "Generic project"
    
    prompt = f"""You are Buddy AI, an autonomous assistant for software development tasks.

## Core Capabilities
- Execute tasks efficiently using available tools
- Provide structured, terminal-friendly responses
- Handle file operations, code execution, and system commands

## Execution Strategy
- **Simple requests**: Execute directly with appropriate tools - be concise
- **Complex requests**: Use task_planner for multi-step coordination
- **Math/calculations**: Answer directly or use code_interpreter briefly

## File Creation Rules
- **"write code"/"create code"**: Use fs_write to create actual files (e.g., script.py, app.py)
- **"show me code"/"example code"**: Display code in terminal only
- **"create file"**: Always use fs_write to create the requested file
- **"save to file"**: Use fs_write to save content to specified file

## Available Tools
- fs_read/fs_write: File operations
- execute_bash: Shell commands  
- code_interpreter: Python execution
- task_planner/task_executor: Complex coordination
- introspect: Quality validation

## Response Format
- Use Markdown for structured output
- Be direct and efficient
- Create files when explicitly or implicitly requested

## System Context
- **OS:** {system_context['os']} {system_context['architecture']}
- **Directory:** {system_context['current_directory']}
- **Project:** {project_context_str}

Execute efficiently and provide clear responses."""
    return prompt


class AnalyzerPrompts:
    """Centralized prompt templates for AgentTaskAnalyzer"""
    
    @staticmethod
    def complexity_analysis(user_input: str) -> str:
        """Prompt for analyzing task complexity"""
        return f"""
                Analyze the complexity of this user request and classify it.
                
                REQUEST: {user_input}
                
                Consider:
                - Number of components/features required
                - Technical complexity and integration needs
                - Time and resource requirements
                - Dependencies and coordination needed
                
                Respond with JSON only:
                {{
                    "complexity": "simple|moderate|complex",
                    "reasoning": "Brief explanation of why this complexity level was chosen"
                }}
                
                Guidelines:
                - simple: Single component, straightforward implementation, minimal dependencies
                - moderate: Multiple components, some integration, moderate complexity
                - complex: Many components, complex integrations, significant coordination needed
                """

    @staticmethod
    def framework_selection(user_input: str, complexity: str) -> str:
        """Prompt for framework selection debate"""
        return f"""
            You are participating in a framework selection debate. Analyze this request and recommend the best frameworks/libraries.
            
            REQUEST: {user_input}
            COMPLEXITY: {complexity}
            
            Debate the following framework categories and recommend ONE specific choice for each relevant category:
            
            CATEGORIES TO CONSIDER:
            - web_framework: Flask vs FastAPI vs Django
            - database: PostgreSQL vs MySQL vs SQLite vs MongoDB
            - ml_framework: scikit-learn vs TensorFlow vs PyTorch
            - container: Docker vs Podman
            - orchestration: Kubernetes vs Docker Swarm
            - ci_cd: GitHub Actions vs GitLab CI vs Jenkins
            - monitoring: Prometheus vs DataDog vs New Relic
            - message_queue: Redis vs RabbitMQ vs Apache Kafka
            - api_docs: Swagger/OpenAPI vs Postman vs Insomnia
            - testing: pytest vs unittest vs nose2
            
            For each relevant category, provide your recommendation with reasoning.
            
            Respond with JSON only:
            {{
                "web_framework": "fastapi",
                "database": "postgresql", 
                "ml_framework": "scikit-learn",
                "reasoning": {{
                    "web_framework": "FastAPI chosen for automatic API docs and async support",
                    "database": "PostgreSQL for ACID compliance and JSON support",
                    "ml_framework": "scikit-learn for rapid prototyping and proven algorithms"
                }}
            }}
            
            Only include categories that are relevant to the request.
            """

    @staticmethod
    def task_decomposition(user_input: str) -> str:
        """Prompt for breaking down complex tasks"""
        return f"""
            Break down this request into detailed tasks for execution using Buddy tools.
            
            REQUEST: {user_input}
            
            NOTE: Framework selection will be handled by a separate task (T000) using debate_agent.
            Your tasks should reference framework selections from T000 where needed.
            
            AVAILABLE BUDDY TOOLS:
            - fs_read: Read files, list directories, search patterns, find files, grep across files
            - fs_write: Create, edit, modify files with diff preview, insert at specific lines
            - execute_bash: Execute bash commands with working directory control and timeout
            - code_interpreter: Execute Python code with visualization support and result capture
            - code_quality: Analyze code quality, detect issues, suggest improvements
            - doc_generator: Generate documentation for code repositories
            - memory_manager: Manage conversation memory and context
            - introspect: Self-analysis and capability assessment
            - debate_agent: Multi-perspective analysis and decision making
            - todo: Task planning and execution management
            
            Create DESCRIPTIVE tasks with atomic action steps. Each task should:
            1. Have detailed description explaining WHY and HOW
            2. Include atomic "actions" array with sub-steps
            3. Use introspect tool for validation where appropriate
            4. NOT include direct solutions in the JSON
            
            Respond with JSON array only:
            [
                {{
                    "id": "T001",
                    "name": "Descriptive task name explaining the purpose",
                    "description": "Detailed explanation of what this task accomplishes, why it's needed, what challenges it addresses, and how it fits into the overall project. Include context about dependencies and expected outcomes.",
                    "complexity": "simple|moderate|complex",
                    "dependencies": ["T001"],
                    "buddy_tools": ["fs_write", "execute_bash", "introspect"],
                    "frameworks": {{"web_framework": "fastapi", "database": "postgresql"}},
                    "actions": [
                        {{
                            "step": 1,
                            "action": "Analyze current environment and requirements",
                            "tool": "introspect",
                            "purpose": "Understand current capabilities and validate prerequisites",
                            "sub_steps": [
                                "Check system capabilities",
                                "Validate tool availability",
                                "Assess resource requirements"
                            ]
                        }},
                        {{
                            "step": 2,
                            "action": "Create project structure",
                            "tool": "fs_write",
                            "purpose": "Establish organized directory layout",
                            "sub_steps": [
                                "Create main directories",
                                "Set up configuration files",
                                "Initialize project metadata"
                            ]
                        }},
                        {{
                            "step": 3,
                            "action": "Validate setup completion",
                            "tool": "introspect",
                            "purpose": "Verify all components are properly configured",
                            "sub_steps": [
                                "Check directory structure",
                                "Validate file permissions",
                                "Confirm setup integrity"
                            ]
                        }}
                    ],
                    "success_criteria": "Detailed criteria for validating task completion using Buddy tools",
                    "expected_outputs": ["specific_file.py", "config.json"]
                }}
            ]
            
            Requirements:
            - Make tasks VERY descriptive with detailed context
            - Include introspect tool for validation steps
            - Create atomic action steps with sub-steps
            - Focus on WHAT to do, not HOW to implement
            - Use realistic Buddy tool combinations
            - Include frameworks field with relevant tools from the recommended list
            """

